In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("everydaycodings/global-news-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'global-news-dataset' dataset.
Path to dataset files: /kaggle/input/global-news-dataset


In [ ]:
import pandas as pd
import numpy as np

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/global-news-dataset/rating.csv
/kaggle/input/global-news-dataset/data.csv
/kaggle/input/global-news-dataset/raw-data.csv


In [ ]:
# --- Imports ---
import pandas as pd
import re
import spacy
from tqdm import tqdm

# Chargement du dataset brut 
raw_csv_path = "/kaggle/input/global-news-dataset/raw-data.csv"
raw_df = pd.read_csv(raw_csv_path)

print("Dataset chargé avec succès.")
print("Colonnes disponibles :", raw_df.columns)
print("Nombre total d'articles :", len(raw_df))

# Étape 1 : Nettoyage et normalisation du texte 
def clean_text(text):
    text = str(text).lower()                                   # conversion en minuscules
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)        # suppression des URLs
    text = re.sub(r'\d+', '', text)                            # suppression des chiffres
    text = re.sub(r'[^\w\s]', '', text)                        # suppression de la ponctuation
    text = re.sub(r'\s+', ' ', text).strip()                   # normalisation des espaces
    return text

print("\nNettoyage du texte en cours...")
raw_df['cleaned_text'] = raw_df['content'].astype(str).apply(clean_text)
print("Nettoyage terminé.")

# Étape 2 : Chargement du modèle spaCy 
print("\nChargement du modèle spaCy (anglais)...")
nlp = spacy.load("en_core_web_sm", disable=["ner"])  # désactivation du NER pour accélérer
print("Modèle chargé.")

# Étape 3 : Tokenization, POS tagging et Lemmatization 
pos_list = []
lemmas_list = []

print("\nApplication du pipeline NLP sur l'ensemble du corpus...")
for doc in tqdm(nlp.pipe(raw_df['cleaned_text'].tolist(), batch_size=100, n_process=2)):
    tokens = [token.text for token in doc]
    pos_tags = [token.pos_ for token in doc]
    lemmas = [token.lemma_ for token in doc]
    tokens_list.append(tokens)
    pos_list.append(pos_tags)
    lemmas_list.append(lemmas)

#  Étape 4 : Ajout des résultats dans le DataFrame 
raw_df['tokens'] = tokens_list
raw_df['pos_tags'] = pos_list
raw_df['lemmas'] = lemmas_list

#  Étape 5 : Sauvegarde du dataset annoté 
output_path = "/content/global_news_preprocessed.csv"
raw_df.to_csv(output_path, index=False, encoding='utf-8')

print("\nPrétraitement terminé. Fichier sauvegardé avec succès.")
print(f"Chemin du fichier sauvegardé : {output_path}")
print("Aperçu du dataset final :")
print(raw_df[['content', 'cleaned_text', 'tokens', 'pos_tags', 'lemmas']].head(2))


/tmp/ipython-input-1785499592.py:9: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(raw_csv_path)


Dataset chargé avec succès.
Colonnes disponibles : Index(['article_id', 'source_id', 'source_name', 'author', 'title',
       'description', 'url', 'url_to_image', 'published_at', 'content',
       'category'],
      dtype='object')
Nombre total d'articles : 933257

Nettoyage du texte en cours...
Nettoyage terminé.

Chargement du modèle spaCy (anglais)...
Modèle chargé.

Application du pipeline NLP sur l'ensemble du corpus...


933257it [1:13:22, 211.99it/s]



Prétraitement terminé. Fichier sauvegardé avec succès.
Chemin du fichier sauvegardé : /content/global_news_preprocessed.csv
Aperçu du dataset final :
                                             content  \
0  LOS ANGELES, CA - JANUARY 09: Adam Neumann spe...   
1  This article was created in partnership with W...   

                                        cleaned_text  \
0  los angeles ca january adam neumann speaks ons...   
1  this article was created in partnership with w...   

                                              tokens  \
0  [los, angeles, ca, january, adam, neumann, spe...   
1  [this, article, was, created, in, partnership,...   

                                            pos_tags  \
0  [PROPN, PROPN, AUX, PROPN, PROPN, PROPN, VERB,...   
1  [DET, NOUN, AUX, VERB, ADP, NOUN, ADP, NOUN, P...   

                                              lemmas  
0  [los, angeles, can, january, adam, neumann, sp...  
1  [this, article, be, create, in, partnership, w...  


In [ ]:
#  Étape 6 : Conversion du texte en TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz

# Initialisation du vectorizer TF-IDF
tfidf = TfidfVectorizer(
    max_features=5000,       # limite du vocabulaire
    min_df=5,                # ignore mots apparaissant dans moins de 5 articles
    max_df=0.8,              # ignore mots apparaissant dans plus de 80 % des articles
    ngram_range=(1, 2),      # unigrams + bigrams
    stop_words='english'     # stopwords intégrées
)

print("\nConversion TF-IDF en cours...")
# Fit et transformation
tfidf_matrix = tfidf.fit_transform(raw_df['cleaned_text'])

# Sauvegarde de la matrice TF-IDF au format sparse (.npz)
tfidf_output_path = "/content/tfidf_matrix.npz"
save_npz(tfidf_output_path, tfidf_matrix)
print(f"TF-IDF généré et sauvegardé : {tfidf_output_path}")

# Sauvegarde du vocabulaire TF-IDF pour référence
vocab_path = "/content/tfidf_vocabulary.txt"
with open(vocab_path, "w") as f:
    for word in tfidf.get_feature_names_out():
        f.write(word + "\n")
print(f"Vocabulaire TF-IDF sauvegardé : {vocab_path}")

print("\nTF-IDF terminé. Dimensions : ", tfidf_matrix.shape)



Conversion TF-IDF en cours...
TF-IDF généré et sauvegardé : /content/tfidf_matrix.npz
Vocabulaire TF-IDF sauvegardé : /content/tfidf_vocabulary.txt

TF-IDF terminé. Dimensions :  (933257, 5000)


partie B


In [ ]:
import pandas as pd
import spacy
from tqdm import tqdm
import os

#  Étape 1 : Chargement du dataset ===
preprocessed_path = "/content/global_news_preprocessed.csv"
df_preprocessed = pd.read_csv(preprocessed_path, low_memory=False)

print("Dataset prétraité chargé avec succès.")
print("Nombre total d'articles :", len(df_preprocessed))

#  Étape 2 : Chargement du modèle spaCy 
nlp_ner = spacy.load("en_core_web_sm")
print("Modèle spaCy avec NER chargé.")

#  Étape 3 : Fonction de traitement par lots 
def process_chunk(df_chunk, chunk_idx, save_dir="/content/ner_chunks"):
    os.makedirs(save_dir, exist_ok=True)
    entities_list = []

    print(f"\nTraitement du lot {chunk_idx} ({len(df_chunk)} articles)...")
    for doc in tqdm(
        nlp_ner.pipe(df_chunk['cleaned_text'].astype(str).tolist(),
                     batch_size=50, n_process=2),
        total=len(df_chunk)
    ):
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        entities_list.append(entities)

    df_chunk['entities_spacy'] = entities_list

    # Sauvegarde du lot partiel
    chunk_path = os.path.join(save_dir, f"ner_chunk_{chunk_idx}.csv")
    df_chunk.to_csv(chunk_path, index=False, encoding='utf-8')
    print(f" Lot {chunk_idx} sauvegardé → {chunk_path}")

#  Étape 4 : Traitement en boucle 
chunk_size = 10000
num_chunks = len(df_preprocessed) // chunk_size + 1

for i in range(num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(df_preprocessed))
    df_chunk = df_preprocessed.iloc[start:end].copy()
    process_chunk(df_chunk, i)

print("\n Tous les lots ont été traités et sauvegardés dans /content/ner_chunks")

#  Étape 5 : Fusion finale 
merged_path = "/content/global_news_with_entities_full.csv"
all_chunks = [pd.read_csv(os.path.join("/content/ner_chunks", f))
              for f in os.listdir("/content/ner_chunks") if f.endswith(".csv")]
df_full = pd.concat(all_chunks, ignore_index=True)
df_full.to_csv(merged_path, index=False, encoding='utf-8')

print(f"\n Fusion terminée : {merged_path}")
print(f"Nombre total d’articles avec entités : {len(df_full)}")


Dataset prétraité chargé avec succès.
Nombre total d'articles : 933257
Modèle spaCy avec NER chargé.

Traitement du lot 0 (10000 articles)...


100%|██████████| 10000/10000 [01:01<00:00, 162.37it/s]


✅ Lot 0 sauvegardé → /content/ner_chunks/ner_chunk_0.csv

Traitement du lot 1 (10000 articles)...


100%|██████████| 10000/10000 [00:59<00:00, 167.82it/s]


✅ Lot 1 sauvegardé → /content/ner_chunks/ner_chunk_1.csv

Traitement du lot 2 (10000 articles)...


...

✅ Lot 91 sauvegardé → /content/ner_chunks/ner_chunk_91.csv

Traitement du lot 92 (10000 articles)...


100%|██████████| 10000/10000 [00:59<00:00, 166.74it/s]


✅ Lot 92 sauvegardé → /content/ner_chunks/ner_chunk_92.csv

Traitement du lot 93 (3257 articles)...


100%|██████████| 3257/3257 [00:20<00:00, 162.07it/s]


✅ Lot 93 sauvegardé → /content/ner_chunks/ner_chunk_93.csv

✅ Tous les lots ont été traités et sauvegardés dans /content/ner_chunks

✅ Fusion terminée : /content/global_news_with_entities_full.csv
Nombre total d’articles avec entités : 933257
